In [33]:
# Imports

import json
import geojson
import geopandas
import pandas as pd
import pathlib
import csv
import copy
import ast
import os
import time
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection

MAP_VERSION = 3
STATS_TYPES = ['Count','Percent','nshft','pshft','growth']


In [34]:
fips_codes = {
 '01': {'abbr': 'AL', 'full_name': 'alabama'},
 '02': {'abbr': 'AK', 'full_name': 'alaska'},
 '04': {'abbr': 'AZ', 'full_name': 'arizona'},
 '05': {'abbr': 'AR', 'full_name': 'arkansas'},
 '06': {'abbr': 'CA', 'full_name': 'california'},
 '08': {'abbr': 'CO', 'full_name': 'colorado'},
 '09': {'abbr': 'CT', 'full_name': 'connecticut'},
 '10': {'abbr': 'DE', 'full_name': 'delaware'},
 '11': {'abbr': 'DC', 'full_name': 'district_of_columbia'},
 '12': {'abbr': 'FL', 'full_name': 'florida'},
 '13': {'abbr': 'GA', 'full_name': 'georgia'},
 '15': {'abbr': 'HI', 'full_name': 'hawaii'},
 '16': {'abbr': 'ID', 'full_name': 'idaho'},
 '17': {'abbr': 'IL', 'full_name': 'illinois'},
 '18': {'abbr': 'IN', 'full_name': 'indiana'},
 '19': {'abbr': 'IA', 'full_name': 'iowa'},
 '20': {'abbr': 'KS', 'full_name': 'kansas'},
 '21': {'abbr': 'KY', 'full_name': 'kentucky'},
 '22': {'abbr': 'LA', 'full_name': 'louisiana'},
 '23': {'abbr': 'ME', 'full_name': 'maine'},
 '24': {'abbr': 'MD', 'full_name': 'maryland'},
 '25': {'abbr': 'MA', 'full_name': 'massachusetts'},
 '26': {'abbr': 'MI', 'full_name': 'michigan'},
 '27': {'abbr': 'MN', 'full_name': 'minnesota'},
 '28': {'abbr': 'MS', 'full_name': 'mississippi'},
 '29': {'abbr': 'MO', 'full_name': 'missouri'},
 '30': {'abbr': 'MT', 'full_name': 'montana'},
 '31': {'abbr': 'NE', 'full_name': 'nebraska'},
 '32': {'abbr': 'NV', 'full_name': 'nevada'},
 '33': {'abbr': 'NH', 'full_name': 'new_hampshire'},
 '34': {'abbr': 'NJ', 'full_name': 'new_jersey'},
 '35': {'abbr': 'NM', 'full_name': 'new_mexico'},
 '36': {'abbr': 'NY', 'full_name': 'new_york'},
 '37': {'abbr': 'NC', 'full_name': 'north_carolina'},
 '38': {'abbr': 'ND', 'full_name': 'north_dakota'},
 '39': {'abbr': 'OH', 'full_name': 'ohio'},
 '40': {'abbr': 'OK', 'full_name': 'oklahoma'},
 '41': {'abbr': 'OR', 'full_name': 'oregon'},
 '42': {'abbr': 'PA', 'full_name': 'pennsylvania'},
 '44': {'abbr': 'RI', 'full_name': 'rhode_island'},
 '45': {'abbr': 'SC', 'full_name': 'south_carolina'},
 '46': {'abbr': 'SD', 'full_name': 'south_dakota'},
 '47': {'abbr': 'TN', 'full_name': 'tennessee'},
 '48': {'abbr': 'TX', 'full_name': 'texas'},
 '49': {'abbr': 'UT', 'full_name': 'utah'},
 '50': {'abbr': 'VT', 'full_name': 'vermont'},
 '51': {'abbr': 'VA', 'full_name': 'virginia'},
 '53': {'abbr': 'WA', 'full_name': 'washington'},
 '54': {'abbr': 'WV', 'full_name': 'west_virginia'},
 '55': {'abbr': 'WI', 'full_name': 'wisconsin'},
 '56': {'abbr': 'WY', 'full_name': 'wyoming'}}

In [35]:
# Input Files
# 2 => "02"
def formatted_state_fips_code(fips_number):
    return f'{fips_number:02d}'

#  block_group_geojson_filename(state_fips_code=46, year=2010)
# tl_2010_46_bg10.json
def block_group_geojson_filename(state_fips_code=1, census_year=2010):
    year_string = str(census_year)
    fips_code_string = formatted_state_fips_code(state_fips_code)
    return f'tl_{year_string}_{fips_code_string}_bg10.json'

# wa_washington_zip_codes_geo.min.json
# def zip_code_geojson_filename(state_fips_code=1):
#     formatted_fips_code = formatted_state_fips_code(state_fips_code)
#     state_abbr = fips_codes[formatted_fips_code]["abbr"].lower()
#     state_full_name = fips_codes[formatted_fips_code]["full_name"]
#     return f'{state_abbr}_{state_full_name}_zip_codes_geo.min.json'


# def zip_geojson_files_for_market(market_data):
#     fips_codes = json.loads(market_data['StateFips'])
#     files = []
#     for fips_code in fips_codes:
#         files.append(zip_code_geojson_filename(fips_code))
#     return files

def bg_geojson_files_for_market(market_data, census_year):
    fips_codes = json.loads(market_data['StateFips'])
    files = []
    for fips_code in fips_codes:
        files.append(block_group_geojson_filename(state_fips_code=int(fips_code), census_year=census_year))
    return files

def read_geo_gson_file(file_path):
    with open(file_path) as f:
        gj = geojson.load(f)
        features = gj['features']
    print(f'Loaded {len(features)} features from {file_path}')
    return  features

def read_geojson_files(file_paths, base_path = "./"):
    features = []
    for file_path in file_paths:
        features += read_geo_gson_file(os.path.join(base_path, file_path))
    return features
    

# def read_market_zip_code_usage(market_file_prefix, base_path="./source_data" ):
#     data_type = "zip"
#     csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
#     data_rows = []
#     with open(csv_file_path, encoding='utf-8-sig') as csvfile:
#         reader = csv.DictReader(csvfile)
#         for row in reader:
#             data_rows.append(row)
#     zip_codes = []
#     for data_row in data_rows:
#         zip_codes.append(data_row['Zip'].zfill(5))
#     print(f'Loaded {len(zip_codes)} zip_codes from {csv_file_path}')
#     return zip_codes
        
def read_market_block_group_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "bg"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    block_groups = []
    for data_row in data_rows:
        block_groups.append(data_row['BG'].zfill(12))
    print(f'Loaded {len(block_groups)} block groups from {csv_file_path}')
    return block_groups
        
# market_file_prefix = 'DallasFortWorth'
# market_prefix= 'dfw'
# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes

def file_type_for_data_type(data_type = "bg"):
    if data_type == "zip": 
        return "Zip"
    if data_type == "bg":
        return "BG"

# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
def mapping_data_file_path(market_file_prefix, base_path="./source_data", data_type = "bg"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_{file_type}_Data.csv'

# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes
def map_settings_data_file_path(market_file_prefix, base_path="./source_data", data_type = "bg"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_Settings_Transposed File_{file_type}.csv'

def published_map_data_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}/data/{market_prefix}_{data_type}_data.json'

def published_map_settings_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_settings_data.json'

def published_map_labels_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_labels.json'

def read_mapping_data(market_file_prefix, base_path='./source_data', data_type='bg'):
    data_rows = []
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')
    return data_rows

def create_zip_features(features, zip_data_rows):
    new_features_to_keep = []
    for data_row in zip_data_rows:
        zip_code = data_row['Zip'].zfill(5)
        # find the associated feature
        for feature in features:
            if feature['properties']["ZCTA5CE10"] == zip_code:
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'Zip':
                        if data_row[key]:
                            if data_row[key] != 'inf':
                                new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_bg_features(features, map_settings_data, bg_data_rows, stats_type):
    new_features_to_keep = []
    for data_row in bg_data_rows:
        block_group = str(data_row['BG']).zfill(12)
        # find the associated feature
        found = False
        for feature in features:
            if feature['properties']["GEOID10"] == block_group:
                found = True
                new_feature = copy.deepcopy(feature)
                properties = new_feature['properties']
                properties.pop("ALAND10",0)
                properties.pop("AWATER10",0)
                properties.pop("BLKGRPCE10",0)
                properties.pop("COUNTYFP10",0)
                properties.pop("FUNCSTAT10",0)
                properties.pop("INTPTLAT10",0)
                properties.pop("INTPTLON10",0)
                properties.pop("MTFCC10",0)
                properties.pop("NAMELSAD10",0)
                properties.pop("STATEFP10",0)
                properties.pop("TRACTCE10",0)
                new_feature['properties'] = properties
                for key in list(data_row.keys())[0:-1]:
                    if key not in  ['BG','LAT','LONG'] and stats_type in key: #and 'Count' in key:
                        if data_row[key] and data_row[key] != 'inf':
                            translated_key = map_settings_data[key]['mapbox_segment']
                            value = ast.literal_eval(data_row[key].replace(',',''))
                            value = int(value * 1000000) / 1000000
                            new_feature['properties'][translated_key] =  value
                        else: 
                            print('exclude', data_row, key)
                new_features_to_keep.append(new_feature)
                break
    #         if found == False:
    #             print(f'failed: {block_group}')
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_geojson(features):
    print(f'Created GeoJson with  {len(features)} features')
    return FeatureCollection(features)

def write_geojson(geojson_data, market_prefix, data_type='bg', stats_type='Count',published_data_base_path = "./public"):
    geojson_string = geojson.dumps(geojson_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{market_prefix}_{stats_type.lower()}.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geojson data to  {published_data_base_path}/{market_prefix}/data/{market_prefix}_{stats_type.lower()}.json')
    
def read_map_settings(market_file_prefix, base_path="./source_data", data_type = "bg"):
    data_rows = []
    csv_file_path = map_settings_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')   
    return data_rows

def write_settings_data(settings_data, market_file_prefix, base_path="./public", data_type = "bg"):
    processed_settings_data = OrderedDict()
    segment_number = 0
    for row in settings_data:
        segment_number += 1
        if row["Display_Data_Type"] == "HH_Count":
            row["Display_Data_Type"] = "HH Count"
        mapbox_segment_key = f's{segment_number}'
        row['mapbox_segment'] = mapbox_segment_key
        processed_settings_data[row['UniqueSeriesString']] = row
    json_string = json.dumps(processed_settings_data, indent = 2, sort_keys=False)
    settings_data_file_path = published_map_settings_path(market_file_prefix,  data_type = data_type, published_data_base_path=base_path)
    f = open(settings_data_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote settings data rows to {settings_data_file_path}') 
    return processed_settings_data
    
# def create_zip_labels(market_file_prefix, base_path="./source_data" , data_type = "bg" , published_data_base_path="./public"):
#     data_rows = []
#     #     csv_file_path = './DFW_Mapping_data.csv'
#     csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "bg")
#     with open(csv_file_path, encoding='utf-8-sig') as csvfile:
#         reader = csv.DictReader(csvfile)
#         for row in reader:
#             data_rows.append(row)

#     features = []
#     for data_row in data_rows:
#         feature =  { 
#             "type": "Feature", 
#             "properties": {
#                 "id": data_row['Zip']
#             },
#             "geometry": { 
#                 "type": "Point", 
#                 "coordinates": [ float(data_row['intptlong']), float(data_row['intptlat']) ]
#             }
#         }

#         features.append(feature)
#     geo_json = {
#         "type": "FeatureCollection",
#         "features": features
#     }
#     json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
#     output_file_path = published_map_labels_path(market_file_prefix,  data_type , published_data_base_path="./public")
#     f = open(output_file_path, "w")
#     f.write(json_string)
#     f.close()
#     print(f'Wrote geojson labels data to  {output_file_path}')

def create_bg_labels(market_file_prefix, base_path="./source_data" , data_type = "bg" , published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "bg")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['BG']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['LONG']), float(data_row['LAT']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(market_file_prefix,  data_type , published_data_base_path=published_data_base_path)
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_geojson_settings(csv_file_path, published_data_base_path = "./public"):

    geospatial_settings = OrderedDict()
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            geospatial_settings[row['MarketName']] = row
    geojson_string = json.dumps(geospatial_settings, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/geospatial_settings.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geospatials settings data to  {published_data_base_path}/geospatial_settings.json')
    
# def create_data_folder(market_file_prefix, published_data_base_path):
#     path = os.path.join(published_data_base_path, market_file_prefix, 'data')
#     pathlib.Path(path).mkdir(parents=True, exist_ok=True)

    
# def create_labels_folder(market_file_prefix, published_data_base_path):
#     path = os.path.join(published_data_base_path, market_file_prefix, 'labels')
#     pathlib.Path(path).mkdir(parents=True, exist_ok=True)
    
# def create_mstds_data_config_file(market_file_prefix, published_data_base_path):
#     config_file_contents = {
#         "username": "stobieb",
#         "tilesetSourceId": f'{market_file_prefix}_bg_data-src',
#         "tilesetSourcePath": f'{market_file_prefix}_bg_data.jsonl',
#         "tilesetId": f'{market_file_prefix}_bg_data',
#         "tilesetName": f'{market_file_prefix}_bg_data'
#     }
#     config_file_path = os.path.join(published_data_base_path, market_file_prefix,'data', 'mts-config.json' )
#     with open(config_file_path, 'w') as config_file:
#         config_file.write(json.dumps(config_file_contents, indent=4))
    

# def create_mstds_labels_config_file(market_file_prefix, published_data_base_path):
#     config_file_contents = {
#         "username": "stobieb",
#         "tilesetSourceId": f'{market_file_prefix}_bg_labels-src',
#         "tilesetSourcePath": f'{market_file_prefix}_bg_labels.jsonl',
#         "tilesetId": f'{market_file_prefix}_bg_labels',
#         "tilesetName": f'{market_file_prefix}_bg_labels'
#     }
#     config_file_path = os.path.join(published_data_base_path, market_file_prefix,'labels','mts-config.json' )
#     with open(config_file_path, 'w') as config_file:
#         config_file.write(json.dumps(config_file_contents, indent=4))
    

    
def create_tileset_data_recipe_file(market_file_prefix, published_data_base_path, stats_type):
    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{market_file_prefix}_{stats_type.lower()}': {
          'source': f'mapbox://tileset-source/stobieb/{market_file_prefix}_{stats_type.lower()}_src',
          'minzoom': 0,
          'maxzoom': 13,
          'tiles': {
            'layer_size': 2500
          }
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{market_file_prefix}_{stats_type.lower()}_recipe.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
    

def create_tileset_labels_recipe_file(market_file_prefix, published_data_base_path):
    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{market_file_prefix}_labels': {
          'source': f'mapbox://tileset-source/stobieb/{market_file_prefix}_labels_src',
          'minzoom': 0,
          'maxzoom': 13,
            
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{market_file_prefix}_labels_recipe.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
        
# def mstds_convert_file(market_file_prefix, published_data_base_path, filetype='data'):
#     recipe_file_path = os.path.join(published_data_base_path, market_file_prefix,'labels','mts-recipe.json' )
#     market_file_prefix, published_data_base_path
    

In [36]:

def process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year):
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows: #[0:10]:
        # if market_data['MarketName'] in ['Texas','DallasFortWorth']:
        if True:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MarketName'].lower()
            market_prefix = market_file_prefix
            #create_data_folder(market_file_prefix,published_data_base_path )
            #create_labels_folder(market_file_prefix,published_data_base_path )
            #create_mstds_data_config_file(market_file_prefix, published_data_base_path)
            #create_mstds_labels_config_file(market_file_prefix, published_data_base_path)
            for stats_type in STATS_TYPES:
                create_tileset_data_recipe_file(market_file_prefix, published_data_base_path,stats_type)
                create_tileset_labels_recipe_file(market_file_prefix, published_data_base_path)
            # Zip Processing     
            # data_type = 'zip'
            # print(f'market: {market_prefix}')
            # geojson_files = zip_geojson_files_for_market(market_data)
            # zip_codes = read_market_zip_code_usage(market_file_prefix, base_path=source_data_base_path )
            # zip_data_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
            # features = read_geojson_files(geojson_files, base_path=geojson_files_base_path)
            # features_to_use = create_zip_features(features, zip_data_rows)
            # new_geojson = create_geojson(features_to_use)
            # write_geojson(new_geojson, market_prefix, data_type, published_data_base_path=published_data_base_path)
            # map_settings_data = read_map_settings(market_file_prefix, base_path=source_data_base_path, data_type = "zip")
            # write_settings_data(map_settings_data, market_prefix, base_path=published_data_base_path, data_type = "zip")
            # create_zip_labels(market_file_prefix, base_path=source_data_base_path , data_type = "zip" , published_data_base_path=published_data_base_path)
            # Block Group Processing
            data_type = 'bg'
            map_settings_data = read_map_settings(market_file_prefix, base_path=source_data_base_path, data_type = "bg")
            map_settings = write_settings_data(map_settings_data, market_prefix, base_path=published_data_base_path, data_type = "bg")
            # print(map_settings)
            geojson_files = bg_geojson_files_for_market(market_data,census_year )
            block_groups = read_market_block_group_usage(market_file_prefix, base_path=source_data_base_path )
            block_group_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
            features = read_geojson_files(geojson_files, base_path=geojson_files_base_path)
            for stats_type in STATS_TYPES:
                features_to_use = create_bg_features(features, map_settings, block_group_rows, stats_type)
                new_geojson = create_geojson(features_to_use)
                write_geojson(new_geojson, market_prefix, data_type, stats_type,published_data_base_path=published_data_base_path)
            create_bg_labels(market_file_prefix, base_path=source_data_base_path , data_type = "bg" , published_data_base_path=published_data_base_path)
            market_count += 1
    # Finished states - create overall settings file    
    create_geojson_settings(csv_file_path, published_data_base_path = "./public")
    print(f'*** COMPLETED {market_count} markets')
    print(completed_markets)
                          
        
# source_data_base_path = "./source_data"
# source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v6_7th_run Brian Stobie/"
# published_data_base_path = "./public"
# geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# census_year = 2010


In [37]:
def create_chart_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        # segment = row['Segment']
        year = row['Year']
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(region,year)
        print(urban, suburban, exurban)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        if region not in data_json.keys():
            data_json[region] = {'displayRegion': display_region, 'data': {}}
        if year not in data_json[region]['data'].keys():
            data_json[region]['data'][year] = {}
        if 'Urban' not in data_json[region]['data'][year].keys():
            data_json[region]['data'][year]['Urban'] = {}
            data_json[region]['data'][year]['SubUrban'] = {}
            data_json[region]['data'][year]['ExUrban'] = {}
        data_json[region]['data'][year]['Urban'] = urban
        data_json[region]['data'][year]['SubUrban'] = suburban
        data_json[region]['data'][year]['ExUrban'] = exurban
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_summary_charts.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_summary_charts.json')



In [38]:
def create_comparison_chart_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        display_region = row['DisplayRegion']
        year = row['Year']
        age_segment=row['Display_Age_Segment']
        income_segment=row['Display_Income_Segment']
        key = f'{region}-{year}-{age_segment}-{income_segment}'
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(key)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        isMetro = row['IsMetro']
        needsCaveat = row['NeedsCaveat']
        data_json[key] = {
            "year": year,
            "ageSegment": age_segment,
            "incomeSegment": income_segment,
            "urban": urban,
            "suburban": suburban,
            "exurban": exurban,
            "isMetro": isMetro,
            "needsCaveat": needsCaveat
        }
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_comparison_charts.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_comparison_charts.json')

In [39]:
def create_data_options_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    year_options_set = set()
    age_segment_options_set = set()
    income_segment_options_set = set()
    region_options_list = []
    for row in chart_data_rows:
        year_options_set.add(row["Year"])
        age_segment_options_set.add(row["Display_Age_Segment"])
        income_segment_options_set.add(row["Display_Income_Segment"])
        region = {
            "region": row["Region"],
            "displayRegion": row["DisplayRegion"],
            "isMetro": row["IsMetro"],
            "needsCaveat": row["NeedsCaveat"],
        }
        region_options_list.append(region)
    year_options_list = list(year_options_set)
    year_options_list.sort()
    age_segment_options_list = list(age_segment_options_set)
    age_segment_options_list.sort()
    income_segment_options_list = list(income_segment_options_set)
    income_segment_options_list.sort()
    region_options = pd.DataFrame(region_options_list).drop_duplicates().to_dict('r')
    dataOptions = {
        "yearOptions": year_options_list,
        'ageSegmentOptions': age_segment_options_list,
        'incomeSegmentOptions': income_segment_options_list,
        "regionOptions": region_options
    }
    data_options_string = json.dumps(dataOptions, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/data_options.json', "w")
    f.write(data_options_string)
    f.close()
    print(f'Wrote data options data to  {published_data_base_path}/data_options.json')


In [40]:
# Set up input and output locations
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# Source csv data path
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v6_7th_run Brian Stobie/"
# Output data path - all of the files in this folder should be uploaded to the cms GeoSpatialTool/v1 folder
published_data_base_path = "./public"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
# Census year for geojson
census_year = 2010

# Process the data
process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year)
# create_chart_json(published_data_base_path)
# create_comparison_chart_json(published_data_base_path)
create_data_options_json(published_data_base_path)


Market Data: 1
Loaded 265 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/v6_7th_run Brian Stobie//abilenetx__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/abilenetx_bg_settings_data.json


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import csv
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v6_7th_run Brian Stobie/"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)
market_count = 0
completed_markets = []
for market_data in market_rows[2:-1]:
    market_file_prefix = market_data['MarketName'].lower()
    # if market_data['MarketName'] in ['Texas','DallasFortWorth']:
    if True:
        for stats_type in STATS_TYPES:
            #print(f'tilesets upload-source stobieb {market_file_prefix}_bg_{stats_type.lower()}_data_src {market_file_prefix}_bg_{stats_type.lower()}_data.json --replace')print(f'tilesets upload-source stobieb {market_file_prefix}_bg_{stats_type.lower()}_data_src {market_file_prefix}_bg_{stats_type.lower()}_data.json --replace')
            upload_source_command = [
                'tilesets',
                'upload-source',
                'stobieb',
                f'{market_file_prefix}_bg_{stats_type.lower()}_data_src',
                f'{market_file_prefix}_bg_{stats_type.lower()}_data.json',
                '--replace'
            ]
            # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
            create_tileset_command = [
                'tilesets',
                'create',
                f'stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data',
                '--recipe', f'{market_file_prefix}_bg_{stats_type.lower()}_recipe.json'            ]
            
            
            # print(f'tilesets publish stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data')
            publish_tileset_command = [
                'tilesets',
                'publish',
                f'stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data'
            ]
            
        print(f'tilesets upload-source stobieb {market_file_prefix}_bg_labels_src {market_file_prefix}_bg_labels.json --replace')
        print(f'tilesets create stobieb.{market_file_prefix}_bg_labels --recipe {market_file_prefix}_labels_recipe.json --name "{market_file_prefix}_bg_labels"')
        # print(f'tilesets publish stobieb.{market_file_prefix}_bg_labels')
            

In [ ]:
a =  {"ALAND10": 267278, "AWATER10": 0, "BLKGRPCE10": "1", 
      
      
      "COUNTYFP10": "003", "FUNCSTAT10": "S", "GEOID10": "250039001001", "INTPTLAT10
      
      ": "+42.4545103", "INTPTLON10": "-073.2493561", "MTFCC10": "G5030", "NAMELSAD10": "Block Group 1", "STATEFP10": "25", "TRACTCE10": "900100"}

In [ ]:
a.pop("ALAND10",0)
a.pop("AWATER10",0)
a.pop("BLKGRPCE10",0)
a.pop("COUNTYFP10",0)
a.pop("FUNCSTAT10",0)
a.pop("INTPTLAT10",0)
a.pop("INTPTLON10",0)
a.pop("MTFCC10",0)
a.pop("NAMELSAD10",0)
a.pop("STATEFP10",0)
a.pop("TRACTCE10",0)
\

In [ ]:
a

In [ ]:
feature = {"geometry": {"coordinates": [[[-73.252424, 42.454116], [-73.252573, 42.453586], [-73.252649, 42.453313], [-73.252756, 42.452939], [-73.25289, 42.45243], [-73.252983, 42.452068], [-73.253156, 42.451533], [-73.252985, 42.451566], [-73.251386, 42.451872], [-73.250079, 42.452057], [-73.249772, 42.452108], [-73.248181, 42.452339], [-73.247938, 42.452368], [-73.246949, 42.452516], [-73.246869, 42.45265], [-73.246869, 42.452745], [-73.24681, 42.452782], [-73.246745, 42.452855], [-73.246709, 42.452962], [-73.246534, 42.453596], [-73.246172, 42.454897], [-73.246012, 42.45537], [-73.245866, 42.455916], [-73.245646, 42.456703], [-73.246754, 42.456868], [-73.248612, 42.457159], [-73.250365, 42.457433], [-73.25057, 42.457181], [-73.250935, 42.456749], [-73.25107, 42.45659], [-73.251254, 42.456346], [-73.251579, 42.455873], [-73.251909, 42.455281], [-73.252182, 42.45478], [-73.252409, 42.45417], [-73.252424, 42.454116]]], "type": "Polygon"}, "properties": {"ALAND10": 267278, "AWATER10": 0, "BLKGRPCE10": "1", "COUNTYFP10": "003", "FUNCSTAT10": "S", "GEOID10": "250039001001", "INTPTLAT10": "+42.4545103", "INTPTLON10": "-073.2493561", "MTFCC10": "G5030", "NAMELSAD10": "Block Group 1", "STATEFP10": "25", "TRACTCE10": "900100"}}

In [ ]:
feature.keys()

In [ ]:
feature['properties'].pop("ALAND10",0)

In [ ]:
feature["properties"]

In [ ]:
def upload_to_mapbox(source_data_base_path,published_data_base_path):
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows: #[0:10]:
        # if market_data['MarketName'] in ['Texas','DallasFortWorth']:
        if True:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MarketName'].lower()
            os.system(f'cd {published_data_base_path}; tilesets upload-source stobieb {market_file_prefix}_bg_data-src {market_file_prefix}_bg_data.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_data --recipe {market_file_prefix}-bg-recipe.json --name "{market_file_prefix}_bg_data"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_data')
            os.system(f'cd {published_data_base_path};  tilesets upload-source stobieb {market_file_prefix}_bg_labels-src {market_file_prefix}_bg_labels.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_labels --recipe {market_file_prefix}-labels-recipe.json --name "{market_file_prefix}_bg_labels"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_labels')

In [ ]:
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v6_7th_run Brian Stobie/"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
upload_to_mapbox(source_data_base_path,published_data_base_path)


In [ ]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}

import subprocess
# tilesets job stobieb.dallasfortworth_bg_data cl9kgdofo000209l52zyy3mdi
process = subprocess.Popen(['tilesets', 'job', 'stobieb.dallasfortworth_bg_data', 'cl9kgdofo000209l52zyy3mdi'],
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)
stdout, stderr = process.communicate()
output = stdout.decode('utf-8').split('\n')
error = stderr.decode('utf-8')
result = json.loads(output[0])



In [ ]:
result

In [ ]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
import subprocess

cwd = os.getcwd()
os.chdir(published_data_base_path)

market_file_prefix = 'austin'
stats_type = 'Count'

def upload_source(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_{stats_type.lower()}_src',
        f'{market_file_prefix}_{stats_type.lower()}.json',
        '--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)

def upload_label_source(market_file_prefix, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_labels_src',
        f'{market_file_prefix}_labels.json',
        #'--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)
    
# upload_source(market_file_prefix, stats_type, published_data_base_path)

In [ ]:
def update_recipe(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    update_recipe_command = [
        'tilesets',
        'update-recipe',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}',
        f'{market_file_prefix}_{stats_type.lower()}_recipe.json',
        
    ]

    print(' '.join(update_recipe_command))
    process = subprocess.Popen(update_recipe_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    
def upload_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}',
        '--recipe', 
        f'{market_file_prefix}_{stats_type.lower()}_recipe.json',
        '--name',
        f'{market_file_prefix}_{stats_type.lower()}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

def upload_label_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}',
        '--recipe', f'{market_file_prefix}_{stats_type.lower()}_recipe.json',
        '--name',
        f'{market_file_prefix}_{stats_type.lower()}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

In [ ]:
# upload_tileset(market_file_prefix, stats_type, published_data_base_path)

In [ ]:
import time
def publish_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    publish_tileset_command = [
        'tilesets',
        'publish',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}'
    ]

    print(' '.join(publish_tileset_command))
    process = subprocess.Popen(publish_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    result = json.loads(output[0])
    print(result)
    get_job_status_command = [
        'tilesets',
        'job',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}',
        result['jobId']
    ]
    state = 'processing'
    while state == 'processing':
        process = subprocess.Popen(get_job_status_command,
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)

        stdout, stderr = process.communicate()
        output = stdout.decode('utf-8').split('\n')
        try:
            result = json.loads(output[0])
        except:
            result = {}
        if 'stage' in result.keys():
            state = result['stage']
        if state != 'success':
            print(f'{state}: sleeping')
            time.sleep(30)
    print('finished')


In [ ]:
# publish_tileset('austin', 'count', published_data_base_path)

In [ ]:
import csv
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v6_7th_run Brian Stobie/"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)
market_count = 0
completed_markets = []
STATS_TYPES = ['Count','Percent','nshft','pshft','growth']
for market_data in market_rows[76:]:
    market_file_prefix = market_data['MarketName'].lower()
    # if market_data['MarketName'] in ['Texas','DallasFortWorth']:
    print('creating tilesets for:', market_file_prefix)
    for stats_type in STATS_TYPES:
        upload_source(market_file_prefix, stats_type, published_data_base_path)
        upload_tileset(market_file_prefix, stats_type, published_data_base_path)
        update_recipe(market_file_prefix, stats_type, published_data_base_path)
        publish_tileset(market_file_prefix, stats_type, published_data_base_path)
        
        

In [ ]:
len('albuquerquesantafe_bg_count_data_src')

In [ ]:
# GreensboroWinstonSalem
# GreensboroWinston

# KingsportBristolJohnsonCity
# KingsportBristol

# ProvidenceNewBedford
# ProvidenceNB

# SacramentoStocktonModesto
# Sacramento

# SanFranciscoOaklandSanJose
# SanFrancisco

# WilkesBarreScranton
# WilkesBarre